In [324]:
# importings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
%matplotlib notebook

In [325]:
#importing raw data
covid_raw = pd.read_csv('../Data/Raw/owid-covid-data.csv')

In [326]:
covid_raw.head(5)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [327]:
print(covid_raw.columns)

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [328]:
covid_new = covid_raw.drop(['total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million',
       'icu_patients_per_million',
       'hosp_patients_per_million',
       'weekly_icu_admissions_per_million',
       'weekly_hosp_admissions_per_million',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'new_vaccinations_smoothed_per_million',
       'new_people_vaccinated_smoothed_per_hundred','excess_mortality_cumulative_per_million', 'total_tests',
        'new_tests',
        'positive_rate',
        'tests_per_case',
        'tests_units',
        'excess_mortality_cumulative_absolute',
        'excess_mortality_cumulative'], axis=1)
# For Test columns Because we already have cases columns for positve tests.

In [329]:
# We want to use data's until there are no new cases and deaths in a country (2023-07-02)
# USA didn't submit new cases since 2023-05-21
covid_new['date'] = pd.to_datetime(covid_new['date'])
covid_new = covid_new[covid_new['date'] < '2023-05-20']

In [330]:
print(covid_new.columns)

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'reproduction_rate', 'icu_patients',
       'hosp_patients', 'weekly_icu_admissions', 'weekly_hosp_admissions',
       'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated',
       'total_boosters', 'new_vaccinations', 'new_vaccinations_smoothed',
       'new_people_vaccinated_smoothed', 'stringency_index',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index', 'population',
       'excess_mortality'],
      dtype='object')


In [331]:
# understandig missing datas:
nan_columns = covid_new.columns[covid_new.isna().any()].tolist()
nan_columns

['continent',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'reproduction_rate',
 'icu_patients',
 'hosp_patients',
 'weekly_icu_admissions',
 'weekly_hosp_admissions',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'new_vaccinations',
 'new_vaccinations_smoothed',
 'new_people_vaccinated_smoothed',
 'stringency_index',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'cardiovasc_death_rate',
 'diabetes_prevalence',
 'female_smokers',
 'male_smokers',
 'handwashing_facilities',
 'hospital_beds_per_thousand',
 'life_expectancy',
 'human_development_index',
 'excess_mortality']

now we are going to understand each column missing data and handle it:

In [332]:
# There are a lot of missing data for countries: England, Hong Kong, Macao, Northern Cyprus, Northern Ireland, Scotland, Taiwan, Wales
missing_data_locations = ['England', 'Hong Kong', 'Macao', 'Northern Cyprus', 'Northern Ireland', 'Scotland', 'Taiwan', 'Wales']
covid_new = covid_new[~covid_new['location'].isin(missing_data_locations) ]

In [333]:
# continent:
covid_new['continent'].isna().sum()

14772

<div dir=rtl >
<font  size=3>
در اینجا متوجه می‌شویم که اون‌هایی که ستون قاره مربوطه نال است، نام قاره در ستون مکان آمده است، حال اول چک کنیم که تعداد مبتلا‌های کشور‌های با خود قاره برابر است یا نه.
</font>
</div>

In [334]:
africa_datas = covid_new[covid_new['continent'] == 'Africa']
africa_continent_datas = covid_new[covid_new['location'] == 'Africa']
africa_datas[africa_datas['date'] == '2024-02-18']['total_cases'].sum() == africa_continent_datas[africa_continent_datas['date'] == '2024-02-18']['total_cases']

Series([], Name: total_cases, dtype: bool)

<div dir=rtl >
<font  size=3>
متوجه می‌شویم که داده‌هایی که ستون قاره آن ها خالی است تنها آمده است کشورها را جمع زده است. پس سطر‌های مربوط را پاک می‌کنیم.
</font>
</div>

In [335]:
covid_new = covid_new.drop(covid_new[covid_new['continent'].isna()].index, axis=0)

In [336]:
covid_new[covid_new['continent'].isna()].count

<bound method DataFrame.count of Empty DataFrame
Columns: [iso_code, continent, location, date, total_cases, new_cases, new_cases_smoothed, total_deaths, new_deaths, new_deaths_smoothed, reproduction_rate, icu_patients, hosp_patients, weekly_icu_admissions, weekly_hosp_admissions, total_vaccinations, people_vaccinated, people_fully_vaccinated, total_boosters, new_vaccinations, new_vaccinations_smoothed, new_people_vaccinated_smoothed, stringency_index, population_density, median_age, aged_65_older, aged_70_older, gdp_per_capita, extreme_poverty, cardiovasc_death_rate, diabetes_prevalence, female_smokers, male_smokers, handwashing_facilities, hospital_beds_per_thousand, life_expectancy, human_development_index, population, excess_mortality]
Index: []

[0 rows x 39 columns]>

In [337]:
# total_cases
covid_new[covid_new['total_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
2,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
3,AFG,Asia,Afghanistan,2020-01-08,NaN,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
4,AFG,Asia,Afghanistan,2020-01-09,NaN,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379176,ZWE,Africa,Zimbabwe,2020-03-17,NaN,0.0,0.0,5.0,0.0,0.0,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379177,ZWE,Africa,Zimbabwe,2020-03-18,NaN,0.0,0.0,5.0,0.0,0.0,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379178,ZWE,Africa,Zimbabwe,2020-03-19,NaN,0.0,0.0,5.0,0.0,0.0,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379179,ZWE,Africa,Zimbabwe,2020-03-20,NaN,0.0,0.0,5.0,0.0,0.0,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN


In [338]:
# we can see that when there is no cases the total_cases is NaN so we replace the NaN with 0:
covid_new['total_cases'] = covid_new['total_cases'].fillna(0)

In [339]:
covid_new[covid_new['total_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality


In [340]:
# new_cases
covid_new[covid_new['new_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality
13554,ARG,South America,Argentina,2020-01-01,0.0,NaN,NaN,NaN,NaN,NaN,...,191.032,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN
13555,ARG,South America,Argentina,2020-01-02,0.0,NaN,NaN,NaN,NaN,NaN,...,191.032,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN
13556,ARG,South America,Argentina,2020-01-03,0.0,NaN,NaN,NaN,NaN,NaN,...,191.032,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN
13557,ARG,South America,Argentina,2020-01-04,0.0,NaN,NaN,NaN,NaN,NaN,...,191.032,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN
34033,BLZ,North America,Belize,2022-04-10,57287.0,NaN,NaN,664.0,8.0,1.143,...,176.957,17.11,NaN,NaN,90.083,1.30,74.62,0.716,405285.0,NaN
92277,ECU,South America,Ecuador,2020-09-13,116451.0,NaN,NaN,10864.0,4140.0,591.429,...,140.448,5.55,2.0,12.3,80.635,1.50,77.01,0.759,18001002.0,38.93
98707,GNQ,Africa,Equatorial Guinea,2022-03-20,15800.0,NaN,NaN,183.0,0.0,0.000,...,202.812,7.78,NaN,NaN,24.640,2.10,58.74,0.592,1674916.0,NaN
135722,GTM,North America,Guatemala,2020-04-05,50.0,NaN,NaN,1.0,0.0,0.000,...,155.898,10.18,NaN,NaN,76.665,0.60,74.30,0.663,17843914.0,-20.96
199308,MWI,Africa,Malawi,2022-11-06,88009.0,NaN,NaN,2685.0,2.0,0.286,...,227.349,3.94,4.4,24.7,8.704,1.30,64.26,0.483,20405318.0,NaN
201392,MDV,Asia,Maldives,2020-03-29,16.0,NaN,NaN,NaN,0.0,0.000,...,164.905,9.19,2.1,55.0,95.803,NaN,78.92,0.740,523798.0,NaN


In [341]:
# It seems that these remaining NaN data for new cases are 0 at the beginning of the pandemic
covid_new['new_cases'] = covid_new['new_cases'].fillna(0)

In [342]:
# making sure that we don't have any problem in total_cases for new_cases we filled.
for country_name in covid_new['location'].unique():
    index_values = covid_new[(covid_new['new_cases'] == 0) & (covid_new['location'] == country_name)]['total_cases']
    index_zero = covid_new[(covid_new['new_cases'] == 0) & (covid_new['location'] == country_name)].index
    if not index_zero.empty:
        index_zero = index_zero.tolist()
        index_zero.pop()
        next_index = [i + 1 for i in index_zero]
        next_cases_value = covid_new.loc[next_index, 'total_cases']
        difference = next_cases_value - index_values
        difference = difference.tolist()
        print(f'{country_name} : {list(filter(lambda x: x != 0 and x is not None and not math.isnan(x), difference))}')
    else:
        print(f'{country_name} : No index with zero new cases found')

Afghanistan : []
Albania : []
Algeria : []
American Samoa : []
Andorra : []
Angola : []
Anguilla : []
Antigua and Barbuda : []
Argentina : []
Armenia : []
Aruba : []
Australia : []
Austria : []
Azerbaijan : []
Bahamas : []
Bahrain : []
Bangladesh : []
Barbados : []
Belarus : []
Belgium : []
Belize : []
Benin : []
Bermuda : []
Bhutan : []
Bolivia : []
Bonaire Sint Eustatius and Saba : []
Bosnia and Herzegovina : []
Botswana : []
Brazil : []
British Virgin Islands : []
Brunei : []
Bulgaria : []
Burkina Faso : []
Burundi : []
Cambodia : []
Cameroon : []
Canada : []
Cape Verde : []
Cayman Islands : []
Central African Republic : []
Chad : []
Chile : []
China : []
Colombia : []
Comoros : []
Congo : []
Cook Islands : []
Costa Rica : []
Cote d'Ivoire : []
Croatia : []
Cuba : []
Curacao : []
Cyprus : []
Czechia : []
Democratic Republic of Congo : []
Denmark : []
Djibouti : []
Dominica : []
Dominican Republic : []
Ecuador : []
Egypt : []
El Salvador : []
Equatorial Guinea : []
Eritrea : []
Eston

In [343]:
# new_cases_smoothed
covid_new[covid_new['new_cases_smoothed'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,NaN,0.0,NaN,...,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379104,ZWE,Africa,Zimbabwe,2020-01-05,0.0,0.0,NaN,NaN,0.0,NaN,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379105,ZWE,Africa,Zimbabwe,2020-01-06,0.0,0.0,NaN,NaN,0.0,NaN,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379106,ZWE,Africa,Zimbabwe,2020-01-07,0.0,0.0,NaN,NaN,0.0,NaN,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN
379107,ZWE,Africa,Zimbabwe,2020-01-08,0.0,0.0,NaN,NaN,0.0,NaN,...,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN


In [344]:
# the same thing as new cases
covid_new['new_cases_smoothed'] = covid_new['new_cases_smoothed'].fillna(0)

In [345]:
# Creating a new column : total_cases_smoothed
covid_new['total_cases_smoothed'] = covid_new['new_cases_smoothed'].cumsum()

In [346]:
# total_deaths
covid_new[covid_new['total_deaths'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality,total_cases_smoothed
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,0.0,NaN,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,0.0,NaN,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,0.0,NaN,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,0.0,NaN,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,0.0,NaN,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379162,ZWE,Africa,Zimbabwe,2020-03-03,0.0,0.0,0.0,NaN,0.0,0.0,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379163,ZWE,Africa,Zimbabwe,2020-03-04,0.0,0.0,0.0,NaN,0.0,0.0,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379164,ZWE,Africa,Zimbabwe,2020-03-05,0.0,0.0,0.0,NaN,0.0,0.0,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379165,ZWE,Africa,Zimbabwe,2020-03-06,0.0,0.0,0.0,NaN,0.0,0.0,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08


In [347]:
covid_new['total_deaths'] = covid_new['total_deaths'].fillna(0)

In [348]:
covid_new[covid_new['total_cases'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality,total_cases_smoothed


In [349]:
# new_deaths
covid_new[covid_new['new_deaths'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality,total_cases_smoothed
13554,ARG,South America,Argentina,2020-01-01,0.0,0.0,0.000,0.0,NaN,NaN,...,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,9.993719e+05
13555,ARG,South America,Argentina,2020-01-02,0.0,0.0,0.000,0.0,NaN,NaN,...,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,9.993719e+05
13556,ARG,South America,Argentina,2020-01-03,0.0,0.0,0.000,0.0,NaN,NaN,...,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,9.993719e+05
13557,ARG,South America,Argentina,2020-01-04,0.0,0.0,0.000,0.0,NaN,NaN,...,5.50,16.2,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,9.993719e+05
41388,BES,North America,Bonaire Sint Eustatius and Saba,2021-10-17,2306.0,102.0,14.571,19.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,77.79,NaN,27052.0,NaN,3.981674e+07
58223,CAN,North America,Canada,2022-06-19,3915721.0,23312.0,3330.286,41017.0,NaN,NaN,...,7.37,12.0,16.6,NaN,2.50,82.43,0.929,38454328.0,5.54,8.390407e+07
66057,CHL,South America,Chile,2023-04-02,5264330.0,14253.0,2036.143,61050.0,NaN,NaN,...,8.46,34.2,41.5,NaN,2.11,80.18,0.851,19603736.0,6.37,9.004861e+07
132920,GLP,North America,Guadeloupe,2020-11-01,7742.0,268.0,38.286,126.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,82.14,NaN,395762.0,18.21,2.985051e+08
213379,MEX,North America,Mexico,2020-01-01,0.0,0.0,0.000,0.0,NaN,NaN,...,13.06,6.9,21.4,87.847,1.38,75.05,0.779,127504120.0,NaN,4.469132e+08
213380,MEX,North America,Mexico,2020-01-02,0.0,0.0,0.000,0.0,NaN,NaN,...,13.06,6.9,21.4,87.847,1.38,75.05,0.779,127504120.0,NaN,4.469132e+08


In [350]:
# It seems that these remaining NaN data for new death are 0 at the beginning of the pandemic
covid_new['new_deaths'] = covid_new['new_deaths'].fillna(0)

In [351]:
# making sure that we have problem in total_deaths for new_deaths we filled.
for country_name in covid_new['location'].unique():
    index_values = covid_new[(covid_new['new_deaths'] == 0) & (covid_new['location'] == country_name)]['total_deaths']
    index_zero = covid_new[(covid_new['new_deaths'] == 0) & (covid_new['location'] == country_name)].index
    if not index_zero.empty:
        index_zero = index_zero.tolist()
        index_zero.pop()
        next_index = [i + 1 for i in index_zero]
        next_cases_value = covid_new.loc[next_index, 'total_deaths']
        difference = next_cases_value - index_values
        difference = difference.tolist()
        print(f'{country_name} : {list(filter(lambda x: x != 0 and x is not None and not math.isnan(x), difference))}')
    else:
        print(f'{country_name} : No index with zero new cases found')

Afghanistan : []
Albania : []
Algeria : []
American Samoa : []
Andorra : []
Angola : []
Anguilla : []
Antigua and Barbuda : []
Argentina : []
Armenia : []
Aruba : []
Australia : []
Austria : []
Azerbaijan : []
Bahamas : []
Bahrain : []
Bangladesh : []
Barbados : []
Belarus : []
Belgium : []
Belize : []
Benin : []
Bermuda : []
Bhutan : []
Bolivia : []
Bonaire Sint Eustatius and Saba : []
Bosnia and Herzegovina : []
Botswana : []
Brazil : []
British Virgin Islands : []
Brunei : []
Bulgaria : []
Burkina Faso : []
Burundi : []
Cambodia : []
Cameroon : []
Canada : []
Cape Verde : []
Cayman Islands : []
Central African Republic : []
Chad : []
Chile : []
China : []
Colombia : []
Comoros : []
Congo : []
Cook Islands : []
Costa Rica : []
Cote d'Ivoire : []
Croatia : []
Cuba : []
Curacao : []
Cyprus : []
Czechia : []
Democratic Republic of Congo : []
Denmark : []
Djibouti : []
Dominica : []
Dominican Republic : []
Ecuador : []
Egypt : []
El Salvador : []
Equatorial Guinea : []
Eritrea : []
Eston

In [352]:
covid_new[covid_new['new_deaths_smoothed'].isna()]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality,total_cases_smoothed
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,0.0,0.0,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,0.0,0.0,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,0.0,0.0,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,0.0,0.0,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,0.0,0.0,0.0,NaN,...,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379104,ZWE,Africa,Zimbabwe,2020-01-05,0.0,0.0,0.0,0.0,0.0,NaN,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379105,ZWE,Africa,Zimbabwe,2020-01-06,0.0,0.0,0.0,0.0,0.0,NaN,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379106,ZWE,Africa,Zimbabwe,2020-01-07,0.0,0.0,0.0,0.0,0.0,NaN,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08
379107,ZWE,Africa,Zimbabwe,2020-01-08,0.0,0.0,0.0,0.0,0.0,NaN,...,1.82,1.6,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,7.652433e+08


In [353]:
covid_new['new_deaths_smoothed'] = covid_new['new_deaths_smoothed'].fillna(0)

In [354]:
# Creating a new column : total_deaths_smoothed
covid_new['total_deaths_smoothed'] = covid_new['new_deaths_smoothed'].cumsum()

In [355]:
# reproduction_rate
covid_new['reproduction_rate'].isna().sum()/len(covid_new['reproduction_rate'])

0.3656930404354588

In [356]:
# # As we can see about 37 percent of data's missing, so we have to choose how we can fill the null cells
# # First we plot box plot for each country:

# sns.set(style="whitegrid")
# fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
# axs = axs.flatten()

# for i, country_name in enumerate(covid_new['location'].unique()):
#     country_data = covid_new[covid_new['location'] == country_name]
#     print(f"{country_name} : {country_data['reproduction_rate'].isna().sum()/len(country_data)}")
#     sns.boxplot(y=country_data['reproduction_rate'].dropna(), ax=axs[i])
#     axs[i].set_title(f'Reproduction Rate: {country_name}', fontsize=6)
#     axs[i].tick_params(axis='x', labelrotation=45)
#     axs[i].set_xlabel('')
#     axs[i].set_ylabel('')

# # Remove extra subplots
# for j in range(len(covid_new['location'].unique()), len(axs)):
#     fig.delaxes(axs[j])

# plt.tight_layout()
# plt.show()


<div dir=rtl >
<font  size=3>
زمانی که به نمودار‌ها نگاه می‌کنیم متوجه می شویم که برای آن‌هایی که داده وجود دارد، قرار دادن میانه ایده بدی نیست، همچنین برای آن‌هایی که داده وجود ندارد مقدار منفی یک خواهیم گذاشت.
</font>
</div>

In [357]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['reproduction_rate'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'reproduction_rate'] = -1
    else:
        median = country_data['reproduction_rate'].median()
        covid_new.loc[covid_new['location'] == country_name,
                      'reproduction_rate'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'reproduction_rate'].fillna(median)
# Check if there is any null left:
covid_new['reproduction_rate'].isna().sum()

0

In [358]:
# # icu_patients
# # We have to check icu_patients for each country individually:
# sns.set(style="whitegrid")
# fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
# axs = axs.flatten()
# for i, country_name in enumerate(covid_new['location'].unique()):
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['icu_patients'].isna().sum()/len(country_data)
#     print(f"{country_name} : {ratio_of_null_data}")
#     sns.boxplot(y=country_data['icu_patients'].dropna(), ax=axs[i])
#     axs[i].set_title(f'icu_patients: {country_name}', fontsize=10)
#     axs[i].tick_params(axis='x', labelrotation=45)
#     axs[i].set_xlabel('')
#     axs[i].set_ylabel('')

# for j in range(len(covid_new['location'].unique()), len(axs)):
#     fig.delaxes(axs[j])

# plt.tight_layout()
# plt.show()
       

<div dir=rtl >
<font  size=3>
برای کشور‌هایی که داده وجود ندارد مقدار -۱ می‌گذاریم و برای بقیه از میانه استفاده می‌کنیم.
</font>
</div>

In [359]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['icu_patients'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'icu_patients'] = -1
    else:
        median_reproduction_rate = country_data['icu_patients'].median()
        covid_new.loc[covid_new['location'] == country_name,
                      'icu_patients'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'icu_patients'].fillna(median_reproduction_rate)
# Check if there is any null left:
covid_new['icu_patients'].isna().sum()

0

In [360]:
# # hosp_patients
# # We do exacly like above:

# sns.set(style="whitegrid")
# fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
# axs = axs.flatten()
# for i, country_name in enumerate(covid_new['location'].unique()):
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['hosp_patients'].isna().sum()/len(country_data)
#     print(f"{country_name} : {ratio_of_null_data}")
#     sns.boxplot(y=country_data['hosp_patients'].dropna(), ax=axs[i])
#     axs[i].set_title(f'hosp_patients: {country_name}', fontsize=10)
#     axs[i].tick_params(axis='x', labelrotation=45)
#     axs[i].set_xlabel('')
#     axs[i].set_ylabel('')

# for j in range(len(covid_new['location'].unique()), len(axs)):
#     fig.delaxes(axs[j])

# plt.tight_layout()
# plt.show()

In [361]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['hosp_patients'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'hosp_patients'] = -1
    else:
        median = country_data['hosp_patients'].median()
        covid_new.loc[covid_new['location'] == country_name,
                      'hosp_patients'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'hosp_patients'].fillna(median)
# Check if there is any null left:
covid_new['hosp_patients'].isna().sum()

0

In [362]:
# # weekly_icu_admissions

# sns.set(style="whitegrid")
# fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
# axs = axs.flatten()
# for i, country_name in enumerate(covid_new['location'].unique()):
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['weekly_icu_admissions'].isna().sum()/len(country_data)
#     print(f"{country_name} : {ratio_of_null_data}")
#     sns.boxplot(y=country_data['weekly_icu_admissions'].dropna(), ax=axs[i])
#     axs[i].set_title(f'weekly_icu_admissions: {country_name}', fontsize=6)
#     axs[i].tick_params(axis='x', labelrotation=45)
#     axs[i].set_xlabel('')
#     axs[i].set_ylabel('')

# for j in range(len(covid_new['location'].unique()), len(axs)):
#     fig.delaxes(axs[j])

# plt.tight_layout()
# plt.show()

In [363]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['weekly_icu_admissions'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'weekly_icu_admissions'] = -1
    else:
        median_reproduction_rate = country_data['weekly_icu_admissions'].median()
        covid_new.loc[covid_new['location'] == country_name,
                      'weekly_icu_admissions'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'weekly_icu_admissions'].fillna(median_reproduction_rate)
# Check if there is any null left:
covid_new['weekly_icu_admissions'].isna().sum()

0

In [364]:
# # weekly_hosp_admissions

# sns.set(style="whitegrid")
# fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
# axs = axs.flatten()
# for i, country_name in enumerate(covid_new['location'].unique()):
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['weekly_hosp_admissions'].isna().sum()/len(country_data)
#     print(f"{country_name} : {ratio_of_null_data}")
#     sns.boxplot(y=country_data['weekly_hosp_admissions'].dropna(), ax=axs[i])
#     axs[i].set_title(f'weekly_hosp_admissions: {country_name}', fontsize=6)
#     axs[i].tick_params(axis='x', labelrotation=45)
#     axs[i].set_xlabel('')
#     axs[i].set_ylabel('')

# for j in range(len(covid_new['location'].unique()), len(axs)):
#     fig.delaxes(axs[j])

# plt.tight_layout()
# plt.show()

In [365]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['weekly_hosp_admissions'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'weekly_hosp_admissions'] = -1
    else:
        median = country_data['weekly_hosp_admissions'].median()
        covid_new.loc[covid_new['location'] == country_name,
                      'weekly_hosp_admissions'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'weekly_hosp_admissions'].fillna(median)
# Check if there is any null left:
covid_new['weekly_hosp_admissions'].isna().sum()

0

In [366]:
# # stringency_index
# sns.set(style="whitegrid")
# fig, axs = plt.subplots(47, 5, figsize=(15, 200), sharex=True, sharey=True)
# axs = axs.flatten()
# for i, country_name in enumerate(covid_new['location'].unique()):
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['stringency_index'].isna().sum()/len(country_data)
#     print(f"{country_name} : {ratio_of_null_data}")
#     sns.boxplot(y=country_data['stringency_index'].dropna(), ax=axs[i])
#     axs[i].set_title(f'stringency_index: {country_name}', fontsize=6)
#     axs[i].tick_params(axis='x', labelrotation=45)
#     axs[i].set_xlabel('')
#     axs[i].set_ylabel('')

# for j in range(len(covid_new['location'].unique()), len(axs)):
#     fig.delaxes(axs[j])

# plt.tight_layout()
# plt.show()

<div dir=rtl >
<font  size=3>
این شاخص که نشان می‌دهد هر کشور چقدر قرنطینه را جدی گرفته است. به نظر بهتر است از مد برای پر کردن داده‌های خالی استفاده کنیم، چرا که معمولا این کشور‌ها معمولا این قوانین را پیاده سازی کرده
</font>
</div>

In [367]:
for country_name in covid_new['location'].unique():
    country_data = covid_new[covid_new['location'] == country_name]
    if (country_data['stringency_index'].isna().sum() / len(country_data)):
        covid_new.loc[covid_new['location'] == country_name, 'stringency_index'] = -1
    else:
        mode = country_data['stringency_index'].mode().tolist()[0]
        covid_new.loc[covid_new['location'] == country_name,
                      'stringency_index'] = covid_new.loc[covid_new['location'] == country_name,
                                                           'stringency_index'].fillna(mode)
# Check if there is any null left:
covid_new['stringency_index'].isna().sum()

0

In [368]:
# population_density
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['population_density'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [369]:
nan_countries = covid_new.groupby('location')['population_density'].transform('count') == 0
covid_new.loc[nan_countries, 'population_density'] = -1
covid_new['population_density'].isna().sum()

0

In [370]:
# median_age
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['median_age'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [371]:
nan_countries = covid_new.groupby('location')['median_age'].transform('count') == 0
covid_new.loc[nan_countries, 'median_age'] = -1
covid_new['median_age'].isna().sum()

0

In [372]:
# aged_65_older
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['aged_65_older'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [373]:
nan_countries = covid_new.groupby('location')['aged_65_older'].transform('count') == 0
covid_new.loc[nan_countries, 'aged_65_older'] = -1
covid_new['aged_65_older'].isna().sum()

0

In [374]:
# aged_70_older
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['aged_70_older'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [375]:
nan_countries = covid_new.groupby('location')['aged_70_older'].transform('count') == 0
covid_new.loc[nan_countries, 'aged_70_older'] = -1
covid_new['aged_70_older'].isna().sum()

0

In [376]:
# gdp_per_capita
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['gdp_per_capita'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [377]:
nan_countries = covid_new.groupby('location')['gdp_per_capita'].transform('count') == 0
covid_new.loc[nan_countries, 'gdp_per_capita'] = -1
covid_new['gdp_per_capita'].isna().sum()

0

In [378]:
# extreme_poverty
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['extreme_poverty'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [379]:
nan_countries = covid_new.groupby('location')['extreme_poverty'].transform('count') == 0
covid_new.loc[nan_countries, 'extreme_poverty'] = -1
covid_new['extreme_poverty'].isna().sum()

0

In [380]:
# cardiovasc_death_rate
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['cardiovasc_death_rate'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [381]:
nan_countries = covid_new.groupby('location')['cardiovasc_death_rate'].transform('count') == 0
covid_new.loc[nan_countries, 'cardiovasc_death_rate'] = -1
covid_new['cardiovasc_death_rate'].isna().sum()

0

In [382]:
# diabetes_prevalence
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['diabetes_prevalence'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [383]:
nan_countries = covid_new.groupby('location')['diabetes_prevalence'].transform('count') == 0
covid_new.loc[nan_countries, 'diabetes_prevalence'] = -1
covid_new['diabetes_prevalence'].isna().sum()

0

In [384]:
# female_smokers
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['female_smokers'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [385]:
nan_countries = covid_new.groupby('location')['female_smokers'].transform('count') == 0
covid_new.loc[nan_countries, 'female_smokers'] = -1
covid_new['female_smokers'].isna().sum()

0

In [386]:
# male_smokers
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['male_smokers'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [387]:
nan_countries = covid_new.groupby('location')['male_smokers'].transform('count') == 0
covid_new.loc[nan_countries, 'male_smokers'] = -1
covid_new['male_smokers'].isna().sum()

0

In [388]:
# handwashing_facilities
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['handwashing_facilities'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [389]:
nan_countries = covid_new.groupby('location')['handwashing_facilities'].transform('count') == 0
covid_new.loc[nan_countries, 'handwashing_facilities'] = -1
covid_new['handwashing_facilities'].isna().sum()

0

In [390]:
# hospital_beds_per_thousand
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['hospital_beds_per_thousand'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [391]:
nan_countries = covid_new.groupby('location')['hospital_beds_per_thousand'].transform('count') == 0
covid_new.loc[nan_countries, 'hospital_beds_per_thousand'] = -1
covid_new['hospital_beds_per_thousand'].isna().sum()

0

In [392]:
# life_expectancy
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['life_expectancy'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [393]:
nan_countries = covid_new.groupby('location')['life_expectancy'].transform('count') == 0
covid_new.loc[nan_countries, 'life_expectancy'] = -1
covid_new['life_expectancy'].isna().sum()

0

In [394]:
# human_development_index
# for country_name in covid_new['location'].unique():
#     country_data = covid_new[covid_new['location'] == country_name]
#     ratio_of_null_data = country_data['human_development_index'].isna().sum()/len(country_data)
#     if ratio_of_null_data != 0:
#         print(f"{country_name} : {ratio_of_null_data}")

In [395]:
nan_countries = covid_new.groupby('location')['human_development_index'].transform('count') == 0
covid_new.loc[nan_countries, 'human_development_index'] = -1
covid_new['human_development_index'].isna().sum()

0

<div dir=rtl >
<font  size=3>
برای ستون‌های زیر به مقادیر خالی با این منطق پر می‌شود که مقادیر خالی از مقدار پر قبلی پر می‌شود. 
</font>
</div>

In [299]:
# total_vaccinations
nan_countries = covid_new.groupby('location')['total_vaccinations'].transform('count') == 0
covid_new.loc[nan_countries, 'total_vaccinations'] = -1
covid_new['total_vaccinations'] = covid_new.groupby('location')['total_vaccinations'].ffill()
covid_new['total_vaccinations'] = covid_new['total_vaccinations'].fillna(0)
covid_new['total_vaccinations'].isna().sum()

0

In [300]:
# people_vaccinated
nan_countries = covid_new.groupby('location')['people_vaccinated'].transform('count') == 0
covid_new.loc[nan_countries, 'people_vaccinated'] = -1
covid_new['people_vaccinated'] = covid_new.groupby('location')['people_vaccinated'].ffill()
covid_new['people_vaccinated'] = covid_new['people_vaccinated'].fillna(0)
covid_new['people_vaccinated'].isna().sum()

0

In [301]:
# people_fully_vaccinated
nan_countries = covid_new.groupby('location')['people_fully_vaccinated'].transform('count') == 0
covid_new.loc[nan_countries, 'people_fully_vaccinated'] = -1
covid_new['people_fully_vaccinated'] = covid_new.groupby('location')['people_fully_vaccinated'].ffill()
covid_new['people_fully_vaccinated'] = covid_new['people_fully_vaccinated'].fillna(0)
covid_new['people_fully_vaccinated'].isna().sum()

0

In [ ]:
# total_boosters
nan_countries = covid_new.groupby('location')['total_boosters'].transform('count') == 0
covid_new.loc[nan_countries, 'total_boosters'] = -1
covid_new['total_boosters'] = covid_new.groupby('location')['total_boosters'].ffill()
covid_new['total_boosters'] = covid_new['total_boosters'].fillna(0)
covid_new['total_boosters'].isna().sum()

In [315]:
# new_vaccinations_smoothed
nan_countries = covid_new.groupby('location')['new_vaccinations_smoothed'].transform('count') == 0
covid_new.loc[nan_countries, 'new_vaccinations_smoothed'] = -1
# Other NaN are for days that we didn't have any vaccine so they should be 0.
covid_new['new_vaccinations_smoothed'] = covid_new['new_vaccinations_smoothed'].fillna(0)
covid_new['new_vaccinations_smoothed'].isna().sum()

0

In [ ]:
# because we have new_vaccinations_smoothed and the data from new_vaccination has lots of NaN, I wil drop this column.
covid_new = covid_new.drop("new_vaccinations", axis=1)

In [320]:
# new_people_vaccinated_smoothed
nan_countries = covid_new.groupby('location')['new_people_vaccinated_smoothed'].transform('count') == 0
covid_new.loc[nan_countries, 'new_people_vaccinated_smoothed'] = -1
# Other NaN are for days that we didn't have any vaccine so they should be 0.
covid_new['new_people_vaccinated_smoothed'] = covid_new['new_vaccinations_smoothed'].fillna(0)
covid_new['new_people_vaccinated_smoothed'].isna().sum()

0

In [321]:
# excess_mortality
nan_countries = covid_new.groupby('location')['excess_mortality'].transform('count') == 0
covid_new.loc[nan_countries, 'excess_mortality'] = -1000
covid_new['excess_mortality'] = covid_new.groupby('location')['excess_mortality'].ffill()
covid_new['excess_mortality'] = covid_new['excess_mortality'].fillna(0)
covid_new['excess_mortality'].isna().sum()

0

In [323]:
covid_new.columns[covid_new.isna().any()].tolist()

[]

In [ ]:
# so we don't have null value
